if you want to `encapsulate the logic for extracting` the data from httpcontext, you can implementing `BindAsync` in your **endpoint handler parameter types**

In [ ]:
public static ValueTask<T> BindAsync(HttpContext context)

public static ValueTask<T> BindAsync(HttpContext context, ParameterInfo parameter)


In [ ]:
public class ProductId
{
    public string Id { get; set; }

    public static ValueTask<ProductId> BindAsync(HttpContext context, ParameterInfo parameter)
    {
        // بایندینگ از مسیر (route) یا query string
        var routeValues = context.Request.RouteValues;
        if (routeValues.TryGetValue(parameter.Name, out var value))
        {
            return new ValueTask<ProductId>(new ProductId { Id = value.ToString() });
        }

        // بایندینگ از query string
        if (context.Request.Query.TryGetValue(parameter.Name, out var queryValue))
        {
            return new ValueTask<ProductId>(new ProductId { Id = queryValue });
        }

        // بایندینگ ناموفق
        return new ValueTask<ProductId>(Task.FromException<ProductId>(new ArgumentException($"Unable to bind {parameter.Name}")));
    }
}

app.MapGet("/product/{id}", async (ProductId id) =>
{
    return $"Received product ID: {id.Id}";
});



In [ ]:
public record SizeDetails(double height, double width)
{
    public static async ValueTask<SizeDetails?> BindAsync(
        HttpContext context)
    {
        using var sr = new StreamReader(context.Request.Body);

        string? line1 = await sr.ReadLineAsync(context.RequestAborted);
        if (line1 is null) { return null; }

        string? line2 = await sr.ReadLineAsync(context.RequestAborted);
        if (line2 is null) { return null; }

        return double.TryParse(line1, out double height)
            && double.TryParse(line2, out double width)
                ? new SizeDetails(height, width)
                : null;
    }
}